In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [29]:
# !pip install matplotlib
# !pip install pandas
# !pip install pyspark
# !pip install databricks-api

In [23]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 
from databricks_api import DatabricksAPI
import requests
import json
import datetime as dt
from hdfs import InsecureClient
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, monotonically_increasing_id, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import *

import csv
import sys
import datetime

In [3]:
JDBC = {
    'url':'jdbc:oracle:thin:@realestate_high?TNS_ADMIN=/home/big/study/db/Wallet_REALESTATE'
    ,'props':{
        'user':'dw_realestate',
        'password':'123qwe!@#QWE'
    }   
}

In [ ]:
#월 리스트 생성
deal_y = ['201'+str(i) for i in range(7,10)]
deal_y.append('2020')
deal_y.append('2021')
deal_y.append('2022')

deal_ymd = []
for year in deal_y:
        for m in range(1,13):
            if m < 10:
                m = str(0) + str(m)
            if year+str(m) > '202209' :
                break
            deal_ymd.append(year+str(m))

deal_ymd=deal_ymd[1:]
print(deal_ymd)

In [12]:
from enum import Enum

# 데이터웨어하우스 ENUM
class DataWarehouse(Enum):
    URL='jdbc:oracle:thin:@realestate_high?TNS_ADMIN=/home/big/study/db/Wallet_REALESTATE'
    PROPS={
        'user':'dw_realestate',
        'password':'123qwe!@#QWE'
    }

In [10]:
# 데이터웨어하우스, 데이터마트에 저장하기 위한 함수
def save_data(config, dataframe, table_name):
    dataframe.write.jdbc(url=config.URL.value,
                        table=table_name,
                        mode='append',
                        properties=config.PROPS.value)

# 데이터웨어하우스, 데이터마트에 덮어쓰기 위한 함수
def overwrite_data(config, dataframe, table_name):
    dataframe.write.jdbc(url=config.URL.value,
                        table=table_name,
                        mode='overwrite',
                        properties=config.PROPS.value)

# 데이터웨어하우스, 데이터마트에서 데이터 가져오기 위한 함수
def find_data(config, table_name):
    return spark.read.jdbc(url=config.URL.value,
                                        table=table_name,
                                        properties=config.PROPS.value)

In [4]:
from pyspark.sql import SparkSession 
spark = SparkSession \
.builder \
.master("local") \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

In [ ]:
#for문으로 매월 데이터 hdfs에서 읽어오기
df_loc = find_data(DataWarehouse, 'LOC')

for i in range(len(deal_ymd)):
    path='realestate_own' + deal_ymd[i] + '.json'
    realestate=spark.read.json(path,encoding='utf-8')

    tmp2 = realestate.first()
    realestate_df=spark.createDataFrame(tmp2[0]) 
    
    df2_realestate=realestate_df.select(
    realestate_df.admin_regn1_name.alias('REGN_CODE1'),
    realestate_df.admin_regn2_name.alias('REGN_CODE2'),
    realestate_df.age .alias('OWNER_AGES'),
    realestate_df.appl_nomprs_num.alias('TOT'),
    realestate_df.bs_ym.alias('RES_DATE'),
    realestate_df.cd_name.alias('PROPERTY_TYPE'),
    realestate_df.enr_no_cls_cd_name.alias('OWNER_CLS'),
    realestate_df.rgs_aim_cd.alias('PURPOSE'),
    realestate_df.sex.alias('OWNER_SEX'),)
    
    #LOC테이블과 join
    cond = [df2_realestate.REGN_CODE1 == df_loc.SIDO, df2_realestate.REGN_CODE2 == df_loc.SIGUNGU]
    # df2_realestate.join(df_loc, cond, 'outer').select(df.name, df3.age).collect()
    df_real_loc=df2_realestate.join(df_loc, cond,'inner').select(df_loc.LOC_CODE.alias('REGN_CODE').cast('int'),df2_realestate.RES_DATE,df2_realestate.PROPERTY_TYPE,df2_realestate.PURPOSE,df2_realestate.OWNER_CLS,df2_realestate.OWNER_SEX,df2_realestate.OWNER_AGES,df2_realestate.TOT.cast('int')).show()

    #DW로 보내기
    df_real_loc.write.jdbc(url=JDBC['url'], table='REALESTATE_OWN', mode='append', properties=JDBC['props'])

In [6]:
path='/project/realestate_own201701.json'
realestate=spark.read.json(path,encoding='utf-8')
type(realestate)
realestate.show()

pyspark.sql.dataframe.DataFrame

+-----------------------------+
|                     dataList|
+-----------------------------+
|[{서울특별시, 강남구, 19세...|
+-----------------------------+



In [25]:
tmp2 = realestate.first()
realestate_df=spark.createDataFrame(tmp2[0]) 


In [26]:
pd_realestate=realestate_df.toPandas()
realestate_df=realestate_df.select(['admin_regn1_name','admin_regn2_name','age',\
                                    'appl_nomprs_num','bs_ym','cd_name','enr_no_cls_cd_name','rgs_aim_cd','sex'])
#칼럼이름 변경
# realestate_df.withColumnRenamed('admin_regn1_name','gender')
# realestate_df.show()

In [36]:
# realestate_df.printSchema()
realestate_df.count()

14720

admin_regn1_name 시도 REGN_CODE

admin_regn2_name 시군구

age 연령대 OWNER_AGES

appl_nomprs_num 명의인수 TOT

bs_ym 접수년월 RES_DATE

cd_name 부동산구분 PROPERTY_TYPE

enr_no_cls_cd 소유자 구분 코드

enr_no_cls_cd_name 소유자 구분 OWNER_CLS

rgs_aim_cd 등기목적 PURPOSE

sex 성별 OWNER_SEX



In [13]:
df_loc = find_data(DataWarehouse, 'LOC')
# df_loc.show()

In [14]:
df_loc=find_data(DataWarehouse, 'LOC')
loc_code = df_loc.select(['SIDO','LOC_CODE']).collect()
df_loc_code = spark.createDataFrame(loc_code)
# df_loc_code.show()

In [16]:
#칼럼명 dw와 동일하게 변경
df2_realestate=realestate_df.select(
    realestate_df.admin_regn1_name.alias('REGN_CODE1'),
    realestate_df.admin_regn2_name.alias('REGN_CODE2'),
    realestate_df.age .alias('OWNER_AGES'),
    realestate_df.appl_nomprs_num.alias('TOT'),
    realestate_df.bs_ym.alias('RES_DATE'),
    realestate_df.cd_name.alias('PROPERTY_TYPE'),
    realestate_df.enr_no_cls_cd_name.alias('OWNER_CLS'),
    realestate_df.rgs_aim_cd.alias('PURPOSE'),
    realestate_df.sex.alias('OWNER_SEX'),)
# df2_realestate.show()

In [17]:
#LOC테이블과 join
cond = [df2_realestate.REGN_CODE1 == df_loc.SIDO, df2_realestate.REGN_CODE2 == df_loc.SIGUNGU]
# df2_realestate.join(df_loc, cond, 'outer').select(df.name, df3.age).collect()
df_real_loc=df2_realestate.join(df_loc, cond,'inner').select(df_loc.LOC_CODE.alias('REGN_CODE').cast('int'),df2_realestate.RES_DATE,df2_realestate.PROPERTY_TYPE,df2_realestate.PURPOSE,df2_realestate.OWNER_CLS,df2_realestate.OWNER_SEX,df2_realestate.OWNER_AGES,df2_realestate.TOT.cast('int')).show()


+---------+--------+-------------+----------+------------------------+---------+----------+---+
|REGN_CODE|RES_DATE|PROPERTY_TYPE|   PURPOSE|               OWNER_CLS|OWNER_SEX|OWNER_AGES|TOT|
+---------+--------+-------------+----------+------------------------+---------+----------+---+
|    31140|  201701|         토지|소유권이전|                    법인|     기타|      기타|131|
|    31140|  201701|         토지|소유권이전|                  내국인|     여자| 70세 이상|  7|
|    31140|  201701|         토지|소유권이전|                  내국인|     여자| 60세~69세| 16|
|    31140|  201701|         토지|소유권이전|                  내국인|     여자| 50세~59세| 30|
|    31140|  201701|         토지|소유권이전|                  내국인|     여자| 40세~49세| 24|
|    31140|  201701|         토지|소유권이전|                  내국인|     여자| 30세~39세|  7|
|    31140|  201701|         토지|소유권이전|                  내국인|     여자| 19세~29세|  2|
|    31140|  201701|         토지|소유권이전|                  내국인|     남자| 70세 이상|  4|
|    31140|  201701|         토지|소유권이전|                  내

In [24]:
df_real_loc.write.jdbc(url=JDBC['url'], \
                        table='REALESTATE_OWN', \
                        mode='append', \
                        properties=JDBC['props'])

AttributeError: 'NoneType' object has no attribute 'write'